<a href="https://colab.research.google.com/github/Han1018/BeatGAN/blob/GenerateMidiAPI/MuseGan_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 導入雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 下載必備套件

In [ ]:
!pip install pypianoroll==0.4.6
!pip install tf-nightly
!pip install gast==0.2.0
!pip install flask-ngrok

     |████████████████████████████████| 5.6 MB 16.6 MB/s 
     |████████████████████████████████| 51 kB 5.9 MB/s 
  Created wheel for pypianoroll: filename=pypianoroll-0.4.6-py3-none-any.whl size=20977 sha256=5757f9c0858c82d941c9635db3e2eb3744d36d875025733ddc3ee40e884f3c8f
  Stored in directory: /root/.cache/pip/wheels/40/ec/3b/4768365c13867b24acf342ae0b91d782b448e2d78039bbde0d
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=07edfbcc99a3db98cb1b3b054d61e96b68ff73105e54a483327126d1096b6698
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pypianoroll pretty-midi
     |████████████████████████████████| 468.2 MB 29 kB/s 
     |████████████████████████████████| 5.6 MB 62.9 MB/s 
     |████████████████████████████████| 2.3 MB 25.0 MB/s 
     |████████████████████████████████| 463 kB 71.6 MB/s 
     |████████████████████████████████| 1.3 MB 37.3 MB/s 
     |███████████

### 載入一代tf

In [ ]:
%tensorflow_version 1.10
import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.10`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


### 更改路徑

In [ ]:
import os
os.chdir("/content/drive/MyDrive/museGan") #更改路徑
dir='/content/drive/MyDrive/museGan'
#os.getcwd() #查看當前路徑

In [ ]:
"""This script performs inference from a trained model."""

import logging
import argparse
from pprint import pformat
import numpy as np
import scipy.stats
import tensorflow as tf
from config import LOGLEVEL, LOG_FORMAT
from data_predict_version import load_data, get_samples
from model import Model
from utils import make_sure_path_exists, load_yaml, update_not_none
LOGGER = logging.getLogger("musegan.inference")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2899968/45929032 bytes (6.3%)6053888/45929032 bytes (13.2%)9191424/45929032 bytes (20.0%)12394496/45929032 bytes (27.0%)15417344/45929032 bytes (33.6%)18489344/45929032 bytes (40.3%)21495808/45929032 bytes (46.8%)24510464/45929032 bytes (53.4%)27754496/45929032 bytes (60.4%)30859264/45929032 bytes (67.2%)33693696/45929032 bytes (73.4%)36470784/45929032 bytes (79.4%)

In [ ]:
def parse_arguments():

    #設定以哪個資料夾下的Model做預測
    predict_folder='exp/accompaniment/piano/'

    """Parse and return the command line arguments."""
    parser = argparse.ArgumentParser()
    parser.add_argument('--result_dir',default=predict_folder,
                        help="Directory where the results are saved.")
    parser.add_argument('--checkpoint_dir',default=predict_folder+'model/',
                        help="Directory that contains checkpoints.")
    parser.add_argument('--params', '--params_file', '--params_file_path', default=predict_folder+'params.yaml',
                        help="Path to the file that defines the "
                             "hyperparameters.")
    parser.add_argument('--config',default=predict_folder+'config.yaml', help="Path to the configuration file.")
    parser.add_argument('--runs', type=int, default="1",
                        help="Times to run the inference process.")
    parser.add_argument('--rows', type=int, default=5,
                        help="Number of images per row to be generated.")
    parser.add_argument('--columns', type=int, default=5,
                        help="Number of images per column to be generated.")
    parser.add_argument('--lower', type=float, default=-2,
                        help="Lower bound of the truncated normal random "
                             "variables.")
    parser.add_argument('--upper', type=float, default=2,
                        help="Upper bound of the truncated normal random "
                             "variables.")
    parser.add_argument('--gpu', '--gpu_device_num', type=str, default="0",
                        help="The GPU device number to use.")
    #args = parser.parse_args()
    # colab改成沒有輸入params，等等用指定的方式指定args
    args = parser.parse_args(args=[])
    return args

In [ ]:
def setup():
    """Parse command line arguments, load model parameters, load configurations
    and setup environment."""
    # Parse the command line arguments
    args = parse_arguments()

    # Load parameters
    # colab改成沒有輸入params，等等用指定的方式指定args
    params = load_yaml(args.params)

    # Load training configurations
    config = load_yaml(args.config)
    update_not_none(config, vars(args))

    # Set unspecified schedule steps to default values
    for target in (config['learning_rate_schedule'], config['slope_schedule']):
        if target['start'] is None:
            target['start'] = 0
        if target['end'] is None:
            target['end'] = config['steps']

    # Make sure result directory exists
    make_sure_path_exists(config['result_dir'])

    # Setup GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = config['gpu']

    return params, config

In [ ]:
def inference_main():
    """Main function."""
    # Setup
    logging.basicConfig(level=LOGLEVEL, format=LOG_FORMAT)
    params, config = setup()
    LOGGER.info("Using parameters:\n%s", pformat(params))
    LOGGER.info("Using configurations:\n%s", pformat(config))

    # ============================== Placeholders ==============================
    placeholder_x = tf.placeholder(
        tf.float32, shape=([None] + params['data_shape']))
    placeholder_z = tf.placeholder(
        tf.float32, shape=(None, params['latent_dim']))
    placeholder_c = tf.placeholder(
        tf.float32, shape=([None] + params['data_shape'][:-1] + [1]))
    placeholder_suffix = tf.placeholder(tf.string)

    # ================================= Model ==================================
    # Create sampler configurations
    sampler_config = {
        'result_dir': config['result_dir'],
        'image_grid': (config['rows'], config['columns']),
        'suffix': placeholder_suffix, 'midi': config['midi'],
        'colormap': np.array(config['colormap']).T,
        'collect_save_arrays_op': config['save_array_samples'],
        'collect_save_images_op': config['save_image_samples'],
        'collect_save_pianorolls_op': config['save_pianoroll_samples']}

    # Build model
    model = Model(params)
    if params.get('is_accompaniment'):
        _ = model(
            x=placeholder_x, c=placeholder_c, z=placeholder_z, mode='train',
            params=params, config=config)
        predict_nodes = model(
            c=placeholder_c, z=placeholder_z, mode='predict', params=params,
            config=sampler_config)
    else:
        _ = model(
            x=placeholder_x, z=placeholder_z, mode='train', params=params,
            config=config)
        predict_nodes = model(
            z=placeholder_z, mode='predict', params=params,
            config=sampler_config)

    # Get sampler op
    sampler_op = tf.group([
        predict_nodes[key] for key in (
            'save_arrays_op', 'save_images_op', 'save_pianorolls_op')
        if key in predict_nodes])

    # ================================== Data ==================================
    if params.get('is_accompaniment'):
        data = load_data(config['data_source'], config['data_filename'])
        print('predicted_data_issssss: ',data.shape)

    # ========================== Session Preparation ===========================
    # Get tensorflow session config
    tf_config = tf.ConfigProto()
    tf_config.gpu_options.allow_growth = True

    # Create saver to restore variables
    saver = tf.train.Saver()

    # =========================== Tensorflow Session ===========================
    with tf.Session(config=tf_config) as sess:

        # Restore the latest checkpoint
        LOGGER.info("Restoring the latest checkpoint.")
        with open(os.path.join(config['checkpoint_dir'], 'checkpoint')) as f:
            checkpoint_name = os.path.basename(
                f.readline().split()[1].strip('"'))
        checkpoint_path = os.path.realpath(
            os.path.join(config['checkpoint_dir'], checkpoint_name))
        print('checkpoint_path is:::::',checkpoint_path)
        saver.restore(sess, checkpoint_path)

        # Run sampler op
        for i in range(config['runs']):
            feed_dict_sampler = {
                placeholder_z: scipy.stats.truncnorm.rvs(
                    config['lower'], config['upper'], size=(
                        (config['rows'] * config['columns']),
                        params['latent_dim'])),
                placeholder_suffix: str(i)}  
            if params.get('is_accompaniment'):
                # sample_x = get_samples(
                #     (config['rows'] * config['columns']), data,
                #     use_random_transpose=config['use_random_transpose'])
                # feed_dict_sampler[placeholder_c] = np.expand_dims(
                #     sample_x[..., params['condition_track_idx']], -1)
                feed_dict_sampler[placeholder_c]=data
                # code in "if(i<1)" all are added py paul
                if(i<1):
                    tmp = feed_dict_sampler[placeholder_c]
                    print(i,tmp.shape)
                    for j in range(24):  
                        print(feed_dict_sampler[placeholder_c].shape)  
                        feed_dict_sampler[placeholder_c]=np.vstack([feed_dict_sampler[placeholder_c],tmp])                 
                    	
                    #print('before:\n',sample_x.shape,'\nAfter:\n', sample_x[..., params['condition_track_idx']].shape)
                    #print('params[\'condition_track_idx\']:',params['condition_track_idx'])
                    print('feed_dict_sampler:\n',feed_dict_sampler[placeholder_c].shape)
            sess.run(sampler_op, feed_dict=feed_dict_sampler)


In [ ]:
from pypianoroll import Multitrack
base='/content/drive/MyDrive/museGan/exp/accompaniment/piano/pianorolls/fake_x_bernoulli_sampling/'
def get_midi():
  midi_file = Multitrack(base+'fake_x_bernoulli_sampling_0.npz')
  midi_file.write(base+'fake_x_0.mid')
  return midi_file

# API 拿midi檔
### flask & ngrok source : https://aishuafei.com/google-colab-flask/
### midi:https://stackoverflow.com/questions/28121776/how-do-i-allow-users-to-download-a-midi-file-with-flask-without-getting-a-0-byte



In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,send_file
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
  midi = get_midi()
  new_file = open(base+'fake_x_0.mid', 'rb')
  return send_file(new_file, mimetype='audio/midi')
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
werkzeug             INFO      * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ad7fc5cfcc34.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Aug/2021 13:18:31] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:18:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:19:09] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:19:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:20:39] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:20:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:20:43] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:20:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:20:49] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:20:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:21:19] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 - - [15/Aug/2021 13:21:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 13:29:53] "GET / HTTP/1.1" 200 -
werkzeug             INFO     127.0.0.1 

In [ ]:
inference_main()

musegan.inference    INFO     Using parameters:
{'beat_resolution': 12,
 'condition_track_idx': 1,
 'data_shape': [4, 48, 84, 5],
 'is_accompaniment': True,
 'is_conditional': True,
 'latent_dim': 128,
 'nets': {'discriminator': 'default', 'generator': 'accompaniment'},
 'use_binary_neurons': False}
musegan.inference    INFO     Using configurations:
{'adam': {'beta1': 0.5, 'beta2': 0.9},
 'batch_size': 64,
 'checkpoint_dir': 'exp/accompaniment/piano/model/',
 'colormap': [[1.0, 0.0, 0.0],
              [1.0, 0.5, 0.0],
              [0.0, 1.0, 0.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.5, 1.0]],
 'columns': 5,
 'config': 'exp/accompaniment/piano/config.yaml',
 'data_filename': 'predict_input_data',
 'data_root': './',
 'data_source': 'npy',
 'evaluate_steps': 100,
 'gan_loss_type': 'wasserstein',
 'gpu': '0',
 'initial_learning_rate': 0.001,
 'learning_rate_schedule': {'end': 50000, 'end_value': 0.0, 'start': 45000},
 'log_loss_steps': 100,
 'lower': -2,
 'midi': {'is_

tensorflow           WARNING  From /content/drive/MyDrive/museGan/model.py:30: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



tensorflow           WARNING  From /content/drive/MyDrive/museGan/model.py:30: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

model                INFO     Building model.
model                INFO     Building training nodes.


tensorflow           WARNING  From /content/drive/MyDrive/museGan/model.py:77: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.



tensorflow           WARNING  From /content/drive/MyDrive/museGan/model.py:78: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.



...

filename: filename
root is :  /content/drive/MyDrive/museGan
/content/drive/MyDrive/museGan
predicted_data_issssss:  (1, 4, 48, 84, 1)


musegan.inference    INFO     Restoring the latest checkpoint.


checkpoint_path is::::: /content/drive/My Drive/museGan/exp/accompaniment/piano/model/model.ckpt-300450
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/museGan/exp/accompaniment/piano/model/model.ckpt-300450


tensorflow           INFO     Restoring parameters from /content/drive/My Drive/museGan/exp/accompaniment/piano/model/model.ckpt-300450


0 (1, 4, 48, 84, 1)
(1, 4, 48, 84, 1)
(2, 4, 48, 84, 1)
(3, 4, 48, 84, 1)
(4, 4, 48, 84, 1)
(5, 4, 48, 84, 1)
(6, 4, 48, 84, 1)
(7, 4, 48, 84, 1)
(8, 4, 48, 84, 1)
(9, 4, 48, 84, 1)
(10, 4, 48, 84, 1)
(11, 4, 48, 84, 1)
(12, 4, 48, 84, 1)
(13, 4, 48, 84, 1)
(14, 4, 48, 84, 1)
(15, 4, 48, 84, 1)
(16, 4, 48, 84, 1)
(17, 4, 48, 84, 1)
(18, 4, 48, 84, 1)
(19, 4, 48, 84, 1)
(20, 4, 48, 84, 1)
(21, 4, 48, 84, 1)
(22, 4, 48, 84, 1)
(23, 4, 48, 84, 1)
(24, 4, 48, 84, 1)
feed_dict_sampler:
 (25, 4, 48, 84, 1)
